In [ ]:
# Monto Google Drive para guardar los archivos
print("Montando Google Drive...")
drive.mount('/content/drive')

# Defino la ruta donde se guardarán las fuentes y el dataset CSV

# Se puede cambiar esta ruta a tu preferencia
OUTPUT_DIR = '/content/ocr_fonts'
print(f"Las fuentes se guardarán en: {OUTPUT_DIR}")

# Creo el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Directorio '{OUTPUT_DIR}' asegurado.")



Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Las fuentes se guardarán en: /content/ocr_fonts
Directorio '/content/ocr_fonts' asegurado.


In [ ]:
import os
import requests
import pandas as pd
from google.colab import drive
from googleapiclient.discovery import build
from tqdm.notebook import tqdm # Para barras de progreso en Colab

In [ ]:

# Se define la ruta donde se guardarán las fuentes y el dataset CSV
OUTPUT_DIR = '/content/ocr_fonts'
print(f"Las fuentes se guardarán en: {OUTPUT_DIR}")

# Creo el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Directorio '{OUTPUT_DIR}' asegurado.")

# Configuración de la API de Google Fonts
GOOGLE_FONTS_API_KEY = "INGRESE SU API_KEY AQUI"

if GOOGLE_FONTS_API_KEY == "TU_CLAVE_API_DE_GOOGLE_FONTS_AQUI":
    raise ValueError("Por favor, reemplaza 'TU_CLAVE_API_DE_GOOGLE_FONTS_AQUI' con tu clave de API de Google Fonts.")

# Construyo el servicio de la API de Google Fonts
service = build('webfonts', 'v1', developerKey=GOOGLE_FONTS_API_KEY)
estilos_deseados = {'regular', 'italic', '700', '900', '200', '700italic', '900italic', '200italic'}
estilos_mapeados = [
    {'codigo': 'regular', 'nombre': 'regular'},
    {'codigo': 'italic', 'nombre': 'italic'},
    {'codigo': '700', 'nombre': 'bold'},
    {'codigo': '900', 'nombre': 'black'},
    {'codigo': '200', 'nombre': 'thin'},
    {'codigo': '700italic', 'nombre': 'bold-italic'},
    {'codigo': '900italic', 'nombre': 'black-italic'},
    {'codigo': '200italic', 'nombre': 'thin-italic'}
]
def get_nombre_estilo(codigo):
    for estilo in estilos_mapeados:
        if estilo['codigo'] == codigo:
            return estilo['nombre']
    return 'desconocido'

# Obtengo y Filtro Fuentes de Google Fonts

print("\nObteniendo lista de fuentes de Google Fonts (esto puede tardar unos segundos)...")
try:
    # Obtengo todas las fuentes disponibles
    request = service.webfonts().list(subset='Latin', sort='POPULARITY')
    response = request.execute()
    all_fonts = response.get('items', [])
    print(f"Total de fuentes disponibles en la API: {len(all_fonts)}")
except Exception as e:
    print(f"Error al conectar con la API de Google Fonts: {e}")
    print("Asegúrate de que tu clave de API sea válida y que tengas conexión a internet.")
    all_fonts = [] # Me aseguro que la lista esté vacía en caso de error

# Filtros para asegurar una buena cobertura y variedad
# Considera las categorías de fuentes
# Se han añadido más categorías para una mayor diversidad.
font_categories = ['sans-serif', 'serif', 'display', 'monospace', 'handwriting']
selected_fonts = []
min_fonts_per_category = 500 # Intentar obtener al menos 500 fuentes por categoría para alcanzar un total de 900

# Itero por categorías para asegurar diversidad
for category in font_categories:
    category_fonts = [f for f in all_fonts if f.get('category') == category]
    # Tomo las primeras `min_fonts_per_category` de cada categoría, o todas si hay menos
    selected_fonts.extend(category_fonts[:min_fonts_per_category])
    print(f"Añadidas {len(category_fonts[:min_fonts_per_category])} fuentes de la categoría '{category}'")

# Si no alcanzo las 900, añadir más de las restantes hasta llegar a la meta (priorizando las más populares)
if len(selected_fonts) < 2000:
    remaining_fonts = [f for f in all_fonts if f not in selected_fonts]
    # Añado el resto de las fuentes hasta alcanzar al menos 900
    selected_fonts.extend(remaining_fonts[:(2000 - len(selected_fonts))])
    print(f"Añadidas {len(selected_fonts)} fuentes en total (buscando al menos 2000).")

if not selected_fonts:
    print("No se encontraron fuentes válidas para descargar. Revisa la configuración de la API o los filtros.")

# Descargo Fuentes y Recopilo Metadatos ---

print("\nIniciando descarga de fuentes y recolección de metadatos...")
font_dataset_records = []

# Uso la libreria tqdm para mostrar una barra de progreso mientras se procesan las fuentes
for font in tqdm(selected_fonts, desc="Procesando fuentes"):
    font_name = font['family']
    category = font.get('category', 'unknown') # Obtengo la categoría

    # Creo una subcarpeta para cada familia de fuente
    font_family_dir = os.path.join(OUTPUT_DIR, font_name.replace(" ", "_")) # Reemplazo espacios por guiones bajos

    # La API proporciona un diccionario 'files' con URLs para cada estilo/peso
    variantes_disponibles = set(font.get('variants', []))
    for estilo in variantes_disponibles: # Ordenamos para una salida consistente
        if estilo in estilos_deseados:
            url = font['files'].get(estilo, 'No disponible')
            if url: # Me aseguraro de que la URL exista
                if '.ttf' in url:
                    extension = '.ttf'
                elif '.otf' in url:
                    extension = '.otf'
                else:
                    continue
                # Construyo el nombre del archivo TTF
                nombre_estilo = get_nombre_estilo(estilo)
                file_name = f"{font_name.replace(' ', '_')}-{nombre_estilo}{extension}"
                file_path = os.path.join(font_family_dir, file_name)
                try:
                    response = requests.get(url, stream=True)
                    response.raise_for_status() # Lanza una excepción si la respuesta no es 200 OK
                    os.makedirs(font_family_dir, exist_ok=True)

                    with open(file_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)

                        # Añado registro al dataset
                        font_dataset_records.append({
                            'font_name': font_name,
                            'font_style': nombre_estilo,
                            'file_path': file_path,
                            'category': category
                        })
                except requests.exceptions.RequestException as e:
                    print(f"Error descargando {font_name} ({estilo}) de {url}: {e}")
                except Exception as e:
                    print(f"Error procesando {font_name} ({estilo}): {e}")

print(f"\nDescarga de fuentes completada. Se descargaron {len(font_dataset_records)} variantes de fuentes.")

# Creo y guardo el Dataset CSV

if font_dataset_records:
    df_fonts = pd.DataFrame(font_dataset_records)
    dataset_csv_path = os.path.join(OUTPUT_DIR, 'fonts_dataset.csv')
    df_fonts.to_csv(dataset_csv_path, index=False)
    print(f"\nDataset CSV guardado en: {dataset_csv_path}")
    print("\nPrimeras 5 filas del dataset:")
    print(df_fonts.sample(20))
else:
    print("\nNo se pudo generar el dataset CSV ya que no se descargó ninguna fuente.")


Las fuentes se guardarán en: /content/ocr_fonts
Directorio '/content/ocr_fonts' asegurado.

Obteniendo lista de fuentes de Google Fonts (esto puede tardar unos segundos)...
Total de fuentes disponibles en la API: 1729
Añadidas 500 fuentes de la categoría 'sans-serif'
Añadidas 339 fuentes de la categoría 'serif'
Añadidas 445 fuentes de la categoría 'display'
Añadidas 44 fuentes de la categoría 'monospace'
Añadidas 245 fuentes de la categoría 'handwriting'
Añadidas 1729 fuentes en total (buscando al menos 2000).

Iniciando descarga de fuentes y recolección de metadatos...


Procesando fuentes:   0%|          | 0/1729 [00:00<?, ?it/s]


Descarga de fuentes completada. Se descargaron 3841 variantes de fuentes.

Dataset CSV guardado en: /content/ocr_fonts/fonts_dataset.csv

Primeras 5 filas del dataset:
                font_name    font_style  \
3186      Share Tech Mono       regular   
859                Andika        italic   
3468                Borel       regular   
2655        Balsamiq Sans          bold   
2073           Hepta Slab          thin   
1920         Libre Bodoni        italic   
1178         Lexend Zetta         black   
367              Urbanist       regular   
1206                 KoHo          bold   
1061  Wix Madefor Display          bold   
3568            Anek Odia       regular   
151             Fira Sans  black-italic   
3442      Comforter Brush       regular   
3726   Noto Sans Linear A       regular   
1656           Noto Serif        italic   
80            Nunito Sans          thin   
1021     Familjen Grotesk       regular   
1060  Wix Madefor Display       regular   
3245          

In [ ]:
# generar archivo zip para decargar

!zip -r /content/fuentes.zip /content/ocr_fonts

Se truncaron las últimas líneas 5000 del resultado de transmisión.
updating: content/ocr_fonts/Ysabeau/Ysabeau-black.ttf (deflated 51%)
updating: content/ocr_fonts/Ysabeau/Ysabeau-thin.ttf (deflated 52%)
updating: content/ocr_fonts/Ysabeau/Ysabeau-thin-italic.ttf (deflated 49%)
updating: content/ocr_fonts/Ysabeau/Ysabeau-bold-italic.ttf (deflated 51%)
updating: content/ocr_fonts/Ysabeau/Ysabeau-bold.ttf (deflated 53%)
updating: content/ocr_fonts/Ysabeau/Ysabeau-italic.ttf (deflated 49%)
updating: content/ocr_fonts/Barlow_Condensed/ (stored 0%)
updating: content/ocr_fonts/Barlow_Condensed/Barlow_Condensed-regular.ttf (deflated 51%)
updating: content/ocr_fonts/Barlow_Condensed/Barlow_Condensed-black.ttf (deflated 53%)
updating: content/ocr_fonts/Barlow_Condensed/Barlow_Condensed-italic.ttf (deflated 49%)
updating: content/ocr_fonts/Barlow_Condensed/Barlow_Condensed-thin.ttf (deflated 51%)
updating: content/ocr_fonts/Barlow_Condensed/Barlow_Condensed-black-italic.ttf (deflated 51%)
updati

In [ ]:
df_fonts.to_csv('fonts_dataset.csv', index=False)